<a href="https://colab.research.google.com/github/Dkepffl/2022-2-ESAA/blob/main/Assignment/Assignment221031_text3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHAPTER 08 텍스트 분석**
___


## **05 감성 분석**
___




### | **감성 분석 소개**
- **감성 분석(Sentiment Analysis)**은 **문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법**이다. 소셜 미디어, 여론조사, 온라인 리뷰 등 다양한 분야에서 활용되고 있다.
- 감성 분석은 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산하는 방법을 사용한다.
  + 감성 지수는 긍정 감정 지수와 부정 감성 지수로 구성되며, 이들의 값을 합산해 긍정 감석/부정 감성을 결정한다.
- 감성 분석은 머신러닝 관점에서 지도학습과 비지도학습 방식으로 나눌 수 있다.
  + 지도 학습은 학습 데이터의 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법이다. 일반적인 텍스트 기반의 분류와 거의 동일한다.
  + 비지도학습은 'Lexicon'이라는 일종의 감성 어휘 사전을 이용한다. 'Lexicon'은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, 이를 이용해 긍정/부정 감성 여부를 판단한다.


### | **지도 학습 기반 감성 분석 실습-IMDB 영화평**
- 먼저 지도학습 기반으로 감성 분석을 수행해보자.
- 유명한 영화 정보사이트인 IMDB의 영화평을 이용한다. 영화평의 텍스트를 분석해 감성 분석 결과가 긍정 또는 부정인지를 예측하는 모델을 만들어보자.
- 우선 캐글에서 [데이터](https://www.kaggle.com/c/word2vec-nlp-tutorial/data;)를 다운받고, 로딩해보자.

In [12]:
import pandas as pd

review_df = pd.read_csv('/content/drive/MyDrive/2022-2 ESAA/Data/IMDB/labeledTrainData.tsv', header=0, sep="\t", quoting=3) # Tab(\t) 문자로 분리된 파일
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- 데이터셋의 피처는 다음과 같다.
  + `id` : 각 데이터의 id
  + `sentiment` : 영화평(review)의 Setiment 결과 값(Target Label). 1은 긍정적, 0은 부정적 평가를 의미한다.
  + `review` : 영화평의 텍스트
- 이번에는 텍스트가 어떻게 구성되어 있는지 `review` 컬럼의 텍스트 값을 하나만 확인해보자.

In [13]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

- HTML 형식에서 추출해 `<br/>` 태그가 여전히 존재한다.  `<br/>` 문자열은 피처로 만들 필요가 없으므로, `replace()`에 `str`을 적용하여,  `<br/>` 태그를 모두 공백으로 바꾼다.
- 또한 영어가 아닌 숫자/특수문자 역시 Sentiment를 위한 피처로는 별 의미가 없어 보이므로, 이들도 모두 공란으로 변경한다.
  + 숫자/특수문자를 찾고 이를 변환할 때는 정규 표현식을 이용한다.
  + 파이썬의 `re` 모듈은 정규 표현식을 지원한다.

In [14]:
import re

# html 태그 <br/>을 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />',' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환 
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )

- 결정값 클래스인 `sentiment` 컬럼을 별도로 추출해 Target값 데이터셋을 만들고, 원본 데이터셋에서 `id`와 `sentiment` 컬럼을 삭제해 피처 데이터셋을 생성한다.
- 그리고 `train_test_split()`을 이용하여, 학습용과 테스트용 데이터셋으로 분리한다.

In [15]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

- 학습용 데이터는 17500개, 테스트용 데이터는 7500개의 리뷰로 구성되었다.
- 이제 영화평(Review) 텍스트를 피처 벡터화한 후, ML 분류 알고리즘을 적용해 예측 성능을 측정하자.
- 앞절에서 설명한 `Pipeline` 객체를 이용해, 두 가지를 한 번에 수행해보자.
  + 먼저 Count 벡터화를 적용해 예측 성능을 측정하고, 다음으로 TF-IDF 벡터화를 적용한다.
  + Classifier는 `LogisticRegression`을 이용한다.
  + 예측 성능 평가는 이진 분류임을 고려해 테스트 데이터셋의 정확도와 ROC-AUC를 모두 측정한다.
  

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

- Count 벡터화

In [17]:
## Count 벡터화

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 Count 벡터화
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1] # roc_auc 계산을 위해 실행

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [18]:
print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred), roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8860, ROC-AUC는 0.9503


- TF-IDF 벡터화

In [19]:
## TF-IDF 벡터화
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1] # roc_auc 계산을 위해 실행

In [20]:
print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred), roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


- TF-IDF 기반 피처 벡터화의 예측 성능이 조금 더 낫다.

### | **비지도학습 기반 감성 분석 소개**
- 비지도 감성 분석은 Lexicon을 기반으로 한다.
- 위의 지도학습 기반 감성 분석은 데이터셋에 레이블 값이 있었다.
- 하지만 많은 감성 분석용 데이터는 레이블 값을 가지고 있지 않다.
- Lexicon은 일반적으로 어휘집을 의미하지만, 여기서는 주로 **감성만을 분석하기 위해 지원하는 감성 어휘 사전**(이하 '감성 사전')이다. 
  + 감성 사전은 긍정(Positive) 혹은 부정(Negative) 감성의 정도를 의미하는 수치를 가지고 있으며, 이를 **감성 지수(Polarity score)**라고 한다.
  + 이 감성 지수는 단어의 위치나 주변 단어, 문맥, POS(Part of Speech) 등을 참고해 결정된다.
  + 감성사전을 구현한 패키지는 대표적으로 NLTK 패키지가 있다.
- 감성 사전을 좀 더 자세히 이해하기 위해 NLP 패키지의 WordNet을 먼저 살펴보자.
  + NLP에서 제공하는 WordNet 모듈은 방대한 영어 어휘 사전이다.
  + WordNe은 단순한 어휘 사전이 아니라 시맨틱 분석을 제공하는 어휘 사전이다. 
    + 시맨틱(Semantic) : 문맥상의 의미
    + 언어학에서는 시맨틱을 표현하기 위해 여러 가지 규칙을 정해왔으며, NLP 패키지는 시맨틱을 프로그램적으로 인터페이스 할 수 있는 다양한 방법을 제공한다.
  + 즉, WordNet은 상황에 따라 다르게 사용되는 단어의 시맨틱 정보를 제공한다. 이때 각각의 품사로 구성된 개별 단어를 Synset(Sets of cognitive synonyms)라는 개념을 이용하여 표현한다. Synset은 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념이다.
- NLTK의 감성 사전은 감성에 대한 훌륭한 사전 역할을 수행하지만, 예측 성능은 별로 좋지 않아, 실제 업무에서는 다른 감성 사전이 주로 적용된다.
  + `SentiWordNet` : NLTK 패키지의 WordNet과 유사하게 감성 단어 전용 WordNet을 구현
  + `VADER` : 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공하기 위한 패키지. 감성 분석 결과가 뛰어나며 비교적 빠른 수행 시간을 보장한다.
  + `Pattern` : 예측 성능 측면에서 가장 주목받는 패키지이나 파이썬 2.X 버전에서만 동작한다.
- 이제 `SentiWordNet`과 `VADER` 감성 사전을 이용하여, 감성 분석을 수행한 뒤, 예측 성능을 지도 학습 기반의 분류와 비교해보자.

### | **SentiWordNet을 이용한 감성 분석**

#### **WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해**
___
- `SentiWordNet`은 `WordNet` 기반의 synset을 이용하므로, 먼저 synset에 대해 알아본 후, `SentiWordNet`을 살펴보자.
- 먼저 `WordNet`을 이용하기 위해, NLTK를 설치한 후, `WordNet` 서브 패키지와 데이터셋을 내려 받아야 한다.
- NLTK의 모든 데이터셋과 패키지를 내려받도록 하자.

In [21]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

- NLTK의 모든 데이터셋을 내려받은 후, `WordNet` 모듈을 Import하여 'present'라는 단어에 대한 Synset을 추출해보자.
- `WordNet`의 `synsets()`는 파라미터로 지정된 단어에 대해 `WordNet`에 등재된 모든 Synset 객체를 바환한다.

In [22]:
from nltk.corpus import wordnet as wn

term = 'present'

# 'present'라는 단어로 wordnet의 synsets 생성
synsets = wn.synsets(term)
print('synsets() 반환 type :', type(synsets))
print('synsets() 반환 값 갯수:', len(synsets))
print('synsets() 반환 값 :', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 갯수: 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- `synsets()` 호출 시 반환되는 것은 여러 개의 Synset 객체를 가지는 리스트이다. 위에서는 총 18개의 서로 다른 semantic을 가지는 synset 객체가 반환되었다.
- `Synset('present.n.01')`과 같이 Synset 객체의 파라미터 `'present.n.01'`은 POS 태그를 나타낸다.
  + present는 의미
  + n은 명사 품사
  + 01은 present가 명사로서 여러 가지 뜻을 가지고 있기 때문에, 이를 구분하기 위한 인덱스이다.
- synset 객체가 가지는 여러 가지 속성을 살펴보자.
- Synset은 POS(Part of Speech, 품사), 정의(Definition), 부명제(Lemma) 등으로 시맨틱 요소를 표현할 수 있따.

In [23]:
for synset in synsets :
    print('##### Synset name : ', synset.name(),'#####')
    print('POS :',synset.lexname())
    print('Definition:',synset.definition())
    print('Lemmas:',synset.lemma_names())

##### Synset name :  present.n.01 #####
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
##### Synset name :  present.n.02 #####
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']
##### Synset name :  present.n.03 #####
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
##### Synset name :  show.v.01 #####
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name :  present.v.02 #####
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
##### Synset name :  stage.v.01 #####
POS : verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represen

- `Synset('present.n.01')`과 `Synset('present.n.02')`는 둘 다 명사지만 서로 다른 의미를 가지고 있다.
  + POS : `noun.time` Vs. `noun.possession`
  + Definition : '시간적인 의미로 현재' Vs. '선물'
- WordNet은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있다.
  + `synset` 객체는 단어 간 유사도를 나타내기 위해, `path_similarity()` 메서드를 제공한다.
  + 이를 이용하여, 'tree', 'lion', 'tiger', 'cat', 'dot' 단어들의 간의 유사도를 살펴보자.


In [24]:
# synset 객체를 단어별로 생성합니다. 
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

In [25]:
entities = [tree , lion , tiger , cat , dog]
similarities = []
entity_names = [ entity.name().split('.')[0] for entity in entities]

In [26]:
# 단어별 synset 들을 iteration 하면서 다른 단어들의 synset과 유사도를 측정
for entity in entities:
    similarity = [ round(entity.path_similarity(compared_entity), 2)  for compared_entity in entities ]
    similarities.append(similarity)
    
# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame형태로 저장 
similarity_df = pd.DataFrame(similarities , columns=entity_names,index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


- `SentiWordNet`은 `WordNet`의 Synset과 유사한 `Senti_Synset` 클래스를 가지고 있으며, `Senti_Synset` 클래스를 리스트 형태로 반환한다.

In [27]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 갯수:', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 갯수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


- `SentiSynset` 객체는 단어의 감성을 나타내는 감성 지수(긍정감성지수/부정감성지수)와, 객관성을 나타내는 객관성 지수를 가지고 있다.
  + 어떤 단어가 전혀 감성적이지 않으면, 객관성 지수는 1이 되고, 감성지수는 긍정/부정 감정 지수 둘 다 0이 된다.
- 'father'과 'fabulous'라는 두 개의 단어의 감성 지수와 객관성 지수를 확인해보자.

In [28]:
import nltk
from nltk.corpus import sentiwordnet as swn

In [29]:
father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


In [30]:
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ',fabulous .pos_score())
print('fabulous 부정감성 지수: ',fabulous .neg_score())

fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


- 'father'는 객관적인 단어로 객관성 지수가 1.0이고 긍정/부정 감성 지수 모두 0이다.
- 반면 'fabulous'는 감성 단어로 긍정 감성 지수가 0.875, 부정 감성 지수가 0.125이다.

#### **SentiWordNet을 이용한 영화 감상평 감성 분석**
___
- 이제 앞의 지도 학습 텍스트 분석 예제에서 사용한 IMDB 영화 감상평 데이터에 대해 `SentiWordNet`의 Lexicon을 이용하여 감성 분석을 진행해보자.
- `SentiWordNet`을 이용해 감성 분석을 수행하는 과정은 다음과 같다.
  1. 문서(Document)를 문장(Setence) 단위로 분해
  2. 다시 문장을 단어(Word) 단위로 토큰화하고, 품사 태깅
  3. 품사 태깅된 단어 기반으로 `synset` 객체와 `senti_synset` 객체 생성
  4. `Senti_synset`에서 긍정/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때, 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정
- `SentiWordNet`을 이용하기 위해 `WordNet`을 이용해 문서를 단어로 토큰화하여 어근을 추출하고 품사 태깅을 적용해야 한다.
- 먼저 품사 태깅을 수행하는 함수를 생성해보자.

In [31]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return 

- 이제 문서를 문장 단위로 분해하고, 다시 단어로 토큰화한 후, 품사 태깅을 한 다음, `SentiSynset` 클래스를 생성하고 감성 지수를 합산하는 함수를 생성해보자.
- 각 단어의 긍정 감성 지수에서 부정 감성 지수를 뺀 값이 0 이상일 경우 긍정 감성, 그렇지 않을 경우 부정 감성으로 예측한다.

In [32]:
# import packages
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [33]:
def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)

    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:

        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word , tag in tagged_sentence:
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue  

            lemma = lemmatizer.lemmatize(word, pos=wn_tag) 
            if not lemma:
                continue

            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
                
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0

- 이제 `swn_polarity(text)` 함수를 IMDB 감상평의 개별 문서에 적용해 긍정/부정 감성 여부를 예측해보자.
- 판다스의 apply lambda 구문을 이용해 `swn_polarity(text)`를 개별 감상평 텍스트에 적용한다. 그리고 그 값을 학습 데이터셋에 새로운 컬럼으로 저장한다.

In [34]:
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

- `SentiWordNet`의 감성 분석 예측 성능을 살펴보자.

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix( y_target, preds))
print("정확도:", np.round(accuracy_score(y_target , preds), 4))
print("정밀도:", np.round(precision_score(y_target , preds),4))
print("재현율:", np.round(recall_score(y_target, preds), 4))

[[7668 4832]
 [3636 8864]]
정확도: 0.6613
정밀도: 0.6472
재현율: 0.7091


### | **VADER을 이용한 감성 분석**
- 이번에는 또 다른 Lexicon인 VADER Lexicon을 살펴보자.
- **VADER**는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon이다.
- `VADER`는 `SentimentIntensityAnalyzer` 클래스를 이용해 쉽게 감성 분석을 제공한다.
- NLTK 서브 모듈로 `SentimentIntensityAnalyzer`을 호출하고 간략하게 IMDB의 감상평 한 개만 감성 분석을 수행해 결과를 살펴보자.


In [36]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


- `VADER`을 이용한 감성 분석 과정
  1. `SentimentIntensityAnalyzer` 객체를 생성
  2. 문서별로 `polarity_scores()`를 호출하여, 감성 점수 계산
  3. 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단
- `SentimentIntensityAnalyzer` 객체의 `polarity_scores()` 메서드는 딕셔너리 형태로 감성 점수를 반환한다.
  - `neg` : 부정 감성 지수
  - `neu` : 중립적인 감성 지수
  - `pos` : 긍정 감성 지수
  - `compound` : `neg`, `neu`, `pos` 점수를 적절히 조합해 -1에서 1 사이의 값으로 감성 지수를 표현한 값
- 이제 `VADER`를 이용해 IMDB 감성 분석을 수행해보자. 입력 파라미터로 영화 감상평 텍스트와 긍정.부정을 결정하는 임계값(thereshold)을 입력 받는 `vader_polarity()` 함수를 생성한다.

In [38]:
def vader_polarity(review,threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환 
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

In [39]:
# apply lambda 식을 이용하여 레코드별로 vader_polarity( )를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply( lambda x : vader_polarity(x, 0.1) )
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

In [40]:
print(confusion_matrix( y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target , vader_preds),4))
print("정밀도:", np.round(precision_score(y_target , vader_preds),4))
print("재현율:", np.round(recall_score(y_target, vader_preds),4))

[[ 6747  5753]
 [ 1858 10642]]
정확도: 0.6956
정밀도: 0.6491
재현율: 0.8514
